# Setup


This is a small project to get financial information in mass and quickly in order to analyze stocks fundamentals.

I understand that I don't follow some "good practices" standards, I'm only an enthusiast trying to get things done with more productivity.

Hope this is useful for you somehow.

Thanks for reading.

In [ ]:
!pip install gspread
!pip install yfinance

In [ ]:
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
import gspread

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
workbook = gc.open('name of workbook')
worksheet = workbook.get_worksheet('index')
verify = worksheet.cell('range to verify if you wrote it correctly') #google sheet index differ from python. 
print(verify)

In [ ]:
import requests
import re


In [ ]:
!pip install pyEX


In [ ]:
import pyEX as p


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

In [ ]:
c = p.Client(api_token='iexCloud provides one for premium clients')

# 1st round

In [ ]:
for i in range ('stock tickers range'):
  v = worksheet.cell(i,1).value #column with stock tickers
  tickers = yf.Ticker(v)
  try:
    #yfinance library
    totrev = tickers.info['totalRevenue']
    name = tickers.info['longName']
    ev = tickers.info['enterpriseValue']
    ebitda = tickers.info['ebitda']
    evrev = tickers.info['enterpriseToRevenue']    
    ico = tickers.quarterly_financials.sum(axis=1).to_dict()['Net Income From Continuing Ops']
    totrdvalue = tickers.quarterly_financials.sum(axis=1).to_dict()['Research Development']
    rdlastqvalue = tickers.quarterly_financials.iloc[0,1] #last Q r&d
    explastq = tickers.quarterly_financials.iloc[-6,1] # Last Q Expenses
    totexp = tickers.quarterly_financials.sum(axis=1).to_dict()['Total Operating Expenses']
    salesmkt = tickers.quarterly_financials.sum(axis=1).to_dict()['Selling General Administrative'] # sales mkt
    #pyEX
    inc = c.incomeStatementDF( v, period='quarter', last=9)
    incc = c.incomeStatement( v, period='quarter', last=9)
    cf = c.cashFlowDF(v, period = 'quarter', last=9)
    incq4 = inc[3]
    incq3 = inc[2]
    incq2 = inc[1]
    incq1 = inc[0]
    netin4 = incq4['netIncome']
    netin3 = incq3['netIncome']
    netin2 = incq2['netIncome']
    netin1 = incq1['netIncome']
    ttmNetin = sum([netin1, netin2, netin3, netin4]) #netIncome
    #grossProfit
    gp0 = (inc[0])['grossProfit']
    gp1 = (inc[1])['grossProfit']
    gp2 = (inc[2])['grossProfit']
    gp3 = (inc[3])['grossProfit']
    gp4 = (inc[4])['grossProfit']
    gp5 = (inc[5])['grossProfit']
    gp6 = (inc[6])['grossProfit']
    gp7 = (inc[7])['grossProfit']
    #revenue
    tr0 = (inc[0])['totalRevenue']
    tr1 = (inc[1])['totalRevenue']
    tr2 = (inc[2])['totalRevenue']
    tr3 = (inc[3])['totalRevenue']
    tr4 = (inc[4])['totalRevenue']
    tr5 = (inc[5])['totalRevenue']
    tr6 = (inc[6])['totalRevenue']
    tr7 = (inc[7])['totalRevenue']
    #sum yr I and F
    gpyr1 = sum([gp0,gp1,gp2,gp3])
    gpyr2 = sum([gp4,gp5,gp6,gp7])
    tryr1 = sum([tr0,tr1,tr2,tr3])
    tryr2 = sum([tr4,tr5,tr6,tr7])
    #gross margin yr I and F
    gmyr1 = gpyr1 / tryr1
    gmyr2 = gpyr2 / tryr2
    #grossmargin yoy
    gmyoy = ((gmyr1-gmyr2)/gmyr2)
    #ebit
    incfinal = inc.iloc[1:5]
    ebitF = incfinal['ebit'].sum()
    incinicial = inc.iloc[5:9]
    ebitI = incinicial['ebit'].sum()
    #depreciation
    finalcf = cf.iloc[1:5]
    depF = finalcf['depreciation'].sum()
    inicf = cf.iloc[5:9]
    depI = inicf['depreciation'].sum()
    #ebitda
    ebitdaF = sum([depF,ebitF])
    ebitdaI = sum([depI,ebitI])
    #total revenue
    rev1 = incc[1]['totalRevenue']
    rev2 = incc[2]['totalRevenue']
    rev3 = incc[3]['totalRevenue']
    rev4 = incc[4]['totalRevenue']
    revttmF = sum([rev1,rev2,rev3,rev4])
    rev5 = incc[5]['totalRevenue']
    rev6 = incc[6]['totalRevenue']
    rev7 = incc[7]['totalRevenue']
    rev8 = incc[8]['totalRevenue']
    revttmI = sum([rev5,rev6,rev7,rev8])
    #ebitda margin
    ebitdamgF = ebitdaF/revttmF
    ebitdamgI = ebitdaI/revttmI
    #ebitda margin yoy
    ebitdamgyoy = (ebitdamgF-ebitdamgI)/ebitdamgI    
    #update all
    worksheet.update_cell(i,2,name)
    worksheet.update_cell(i,28,ev)
    worksheet.update_cell(i,34,totrev)
    worksheet.update_cell(i,35,ttmNetin) 
    worksheet.update_cell(i,36,ico)
    worksheet.update_cell(i,37,gmyr1)
    worksheet.update_cell(i,38,totrdvalue)
    worksheet.update_cell(i,39,rdlastqvalue)
    worksheet.update_cell(i,40,salesmkt)
    worksheet.update_cell(i,41,totexp)
    worksheet.update_cell(i,42,explastq)
    worksheet.update_cell(i,43,ebitda)
    worksheet.update_cell(i,48,gmyoy)
    worksheet.update_cell(i,49,ebitdamgyoy)
    worksheet.update_cell(i,71,evrev)
  except:  
    continue

# 2nd round - cash and debt

In [ ]:
for i in range ('stock tickers range'):
  v = worksheet.cell(i,1).value #column with stock tickers
  try:
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-summary"
    querystring = {"symbols":v}
    headers = {
      "X-RapidAPI-Host": "seeking-alpha.p.rapidapi.com",
      "X-RapidAPI-Key": "for clients only"
      }
    response = requests.request("GET", url, headers=headers, params=querystring)
    texto = response.text
    lista = re.split(',',texto)
    cash = re.split(':',lista[1])[2]
    debt = re.split(':',lista[-8])[1]
    worksheet.update_cell(i,45,cash)
    worksheet.update_cell(i,46,debt)
  except:
    continue